<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn import metrics
!pip install talos
import talos

In [109]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [110]:
train_df.shape

(7613, 2)

In [111]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
#x_train = train_df['text']
#y_train = train_df['target']

In [112]:
x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(6090,)

In [113]:
#experiment_data = t.data
#experiment_data.head()

In [114]:
#experiment_data['val_accuracy'].idxmax()

In [115]:
#experiment_data.iloc[205]

In [116]:
#vectorizer = CountVectorizer()
#vectorizer.fit(train_df['text'])

#data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [117]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

#x_train, x_test, y_train, y_test = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)
def modelo(x_train,y_train,x_test,y_test,params):
  tokenizer = Tokenizer(num_words=2000)
  tokenizer.fit_on_texts(x_train)

  train_sequences = tokenizer.texts_to_sequences(x_train)
  validation_sequences = tokenizer.texts_to_sequences(x_validation)
  train_data = pad_sequences(train_sequences, maxlen=100)
  validation_data = pad_sequences(validation_sequences, maxlen=100)


  #input_dim = x_train.shape[1]

  best_model = Sequential()
  best_model.add(layers.Embedding(2000, params['output_dim'], input_length=100))
  best_model.add(Conv1D(params['neuron_units'], params['conv1kernel'], params['conv1activation']))
  best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(MaxPooling1D(5))
  #best_model.add(layers.Dropout(0.25))
  #best_model.add(Flatten())
  #best_model.add(LSTM(64))
  best_model.add(layers.Dense(params['neuron_units'] ,params['dense1activation']))
  #best_model.add(layers.Dense(128 ,activation='relu'))
  best_model.add(layers.Dense(1,activation='sigmoid'))
  #best_model.add(layers.Dense(1,activation = 'sigmoid'))
  best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = best_model.fit(train_data,y_train,epochs = params['epochs'],verbose = 1,validation_data = (validation_data,y_validation),batch_size=params['batch size'])
  return out,best_models

#loss, accuracy = best_model.evaluate(train_data, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = best_model.evaluate(validation_data, y_validation, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

p = {
    'neuron units': [20,32,64,128,256,512],
    'conv1kernel' : [2,3,4,5,6,7,8,9,10],
    'conv1activation' : ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'dense1activation' : ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'epochs' : [1,2,3,4,5,6,7,10,20,100],
    'batch size': [5,100,5],
    'output_dim': [20,32,64,128,256,512]
}

t = talos.Scan(x_train, y_train, params = p, model = modelo, experiment_name = 'Convo hyperparam tuning')

KeyError: ignored

In [ ]:
prediction=best_model.predict_classes(validation_data, batch_size=50)
score = roc_auc_score(y_validation, prediction)
score

In [ ]:
accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

In [ ]:
#Reentrena con el set completo y realiza predicción.

#set_sequences = tokenizer.texts_to_sequences(train_df['text'])
#set_data = pad_sequences(set_sequences, maxlen=100)

#out = best_model.fit(set_data,train_df['target'],epochs = 5,verbose = 1,batch_size=62)

#out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
#data_test = vectorizer.transform(test_df['text']).toarray()
test_prediction = tokenizer.texts_to_sequences(test_df['text'])
prediction_data = pad_sequences(test_prediction, maxlen=100)

prediction2=best_model.predict_classes(prediction_data, batch_size=512)
prediction2

In [ ]:
# Genera archivo submit.

test_df['target'] = prediction2
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')